# Markov Chain Monte Carlo

Markov Chain Monte Carlo (MCMC) is another class of techniques to perform interence by sampling directly from the posterior distribution. In this tutorial we will showcase how to use [emcee](https://emcee.readthedocs.io/en/stable/), an *ensemble sampler*. Ensemble samplers uses a set of walkers that move around the parameter space generally spending more time in high probablity regions. Several types of moves are implemented in `emcee` and can be mixed together to form a probabilistic mixture, see [emcee moves documentation](https://emcee.readthedocs.io/en/stable/user/moves/). Accepting or rejecting the move is based on the [Metropolis algorithm](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm). 

In [ ]:

parser = argparse.ArgumentParser(description='emcee fitter')
parser.add_argument('--cfgfile', type=str, default='', help='Config file name')
parser.add_argument('--nwalkers', type=int, default=32, help='Number of walkers. Default 32')
parser.add_argument('--burnin', type=int, default=100, help='Number of burn-in steps. Default 100')
parser.add_argument('--nsamples', type=int, default=1000, help='Number of samples. Default 1000')
parser.add_argument('--overwrite', action='store_true', help='Overwrite existing output file if exists')
parser.add_argument('--accelerator', type=str, default='', help='Force use of given "accelerator" ~ [gpu, mpi, mpigpu, gpumpi]. Default "" = cpu')
parser.add_argument('--ofolder', type=str, default='mcmc', help='Output folder name. Default "mcmc"')
parser.add_argument('--ofile', type=str, default='mcmc.h5', help='Output file name. Default "mcmc.h5"')
parser.add_argument('--corner_ofile', type=str, default='corner.png', help='Corner plot output file name. Default "corner.png"')
parser.add_argument('--seed', type=int, default=42, help='RNG seed for consistent walker random initialization. Default 42')
parser.add_argument('--yaml_override', type=str, default={}, help='Path to YAML file containing parameter overrides. Default "" = no override')

cfgfile = f'{REPO_HOME}/tests/samples/SIMPLE_EXAMPLE/fit.cfg'

args = parser.parse_args(sys.argv[1:])

yaml_override = {}
if args.yaml_override != {}:
    with open(args.yaml_override, 'r') as f:
        yaml_override = yaml.safe_load(f)

cfgfile  = args.cfgfile if 'cfgfile' not in yaml_override else yaml_override['cfgfile']
ofolder  = args.ofolder if 'ofolder' not in yaml_override else yaml_override['ofolder']
ofile    = args.ofile if 'ofile' not in yaml_override else yaml_override['ofile']
corner_ofile = args.corner_ofile if 'corner_ofile' not in yaml_override else yaml_override['corner_ofile']
nwalkers = args.nwalkers if 'nwalkers' not in yaml_override else yaml_override['nwalkers']
burnIn   = args.burnin if 'burnin' not in yaml_override else yaml_override['burnin']
nsamples = args.nsamples if 'nsamples' not in yaml_override else yaml_override['nsamples']
overwrite_ofile = args.overwrite if 'overwrite' not in yaml_override else yaml_override['overwrite']
seed     = args.seed if 'seed' not in yaml_override else yaml_override['seed']
params_dict = {} if 'params_dict' not in yaml_override else yaml_override['params_dict']
moves_mixture = [ emcee.moves.StretchMove() ] if 'moves_mixture' not in yaml_override else createMovesMixtureFromDict(yaml_override['moves_mixture'])

assert( cfgfile != '' ), 'You must specify a config file'

print("\n ====================================================================================")
print(f" cfgfile: {cfgfile}")
print(f" ofolder: {ofolder}")
print(f" ofile: {ofile}")
print(f" corner_ofile: {corner_ofile}")
print(f" nwalkers: {nwalkers}")
print(f" burnIn: {burnIn}")
print(f" nsamples: {nsamples}")
print(f" overwrite_ofile: {overwrite_ofile}")
print(f" seed: {seed}")
print(f" params_dict: {params_dict}")
print(f" moves_mixture: {moves_mixture}")
print(" ====================================================================================\n")

############## PREPARE FOR SAMPLER ##############
assert( os.path.exists(cfgfile) ), 'Config file does not exist at specified path'
if os.path.isfile(f'{ofolder}/{ofile}') and overwrite_ofile:
    os.system(f'rm {ofolder}/{ofile}')
    print("Overwriting existing output file!")
os.system(f'mkdir -p {ofolder}')

############## SET ENVIRONMENT VARIABLES ##############
REPO_HOME     = os.environ['REPO_HOME']

################### LOAD LIBRARIES ##################
USE_MPI, USE_GPU, RANK_MPI = atiSetup.setup(globals(), args.accelerator)

############## LOAD CONFIGURATION FILE ##############
parser = ConfigFileParser(cfgfile)
cfgInfo: ConfigurationInfo = parser.getConfigurationInfo()
cfgInfo.display()

############## REGISTER OBJECTS FOR AMPTOOLS ##############
AmpToolsInterface.registerAmplitude( Zlm() )
AmpToolsInterface.registerAmplitude( BreitWigner() )
AmpToolsInterface.registerAmplitude( Piecewise() )
AmpToolsInterface.registerAmplitude( PhaseOffset() )
AmpToolsInterface.registerAmplitude( TwoPiAngles() )
AmpToolsInterface.registerDataReader( DataReader() )
AmpToolsInterface.registerDataReader( DataReaderFilter() )

ati = AmpToolsInterface( cfgInfo )
LoadParametersSampler = LoadParameters()
LoadParametersSampler.load_cfg( cfgInfo )

############## RUN MCMC ##############
np.random.seed(seed)

results = perform_mcmc(ati,
                        LoadParametersSampler,
                        ofolder   = ofolder,
                        ofile     = ofile,
                        nwalkers  = nwalkers,
                        burnIn    = burnIn,
                        nsamples  = nsamples,
                        params_dict   = params_dict,
                        moves_mixture = moves_mixture,
                        )
elapsed_fit_time = results['elapsed_fit_time']
draw_corner(results, ofolder, corner_ofile)

print(f"Fit time: {elapsed_fit_time} seconds")
print(f"Total time: {time.time() - start_time} seconds")